In [ ]:
## import necessary libraries
import numpy as np
import pandas as pd

## read the raw data file
df = pd.read_csv('/Users/bruce/Desktop/STATS 302/raw_data/modified_Diabetes_02.csv')

# df.drop(df[df['Diabetes_012'] == 1].index, inplace=True)

# drop duplicates
df.drop_duplicates(inplace=True)

# convert data types
df[['Diabetes_012','GenHlth', 'Age', 'Education','Income']] = df[['Diabetes_012','GenHlth', 'Age', 'Education','Income']].astype(int)

df.info()

In [ ]:
#slice the dataframe for feature and label
df_copy = df.copy(deep=True)
X, y = df_copy.iloc[:,1:], df_copy.iloc[:,0]
print(X.shape)
print(y.shape)

#get dummies
def get_dummies(X):

    # Identify discrete columns with more than 2 categories
    discrete_columns = [col for col in X.columns if X[col].nunique() > 2 and X[col].dtype != 'float64']

    # Create dummy variables for these columns
    df_with_dummies = pd.get_dummies(X, columns=discrete_columns, drop_first=True)

    return df_with_dummies
X = get_dummies(X)
X

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt

## Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
## Standard scaling
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_scaled = scalar.fit_transform(X_train)
X_scaled_test = scalar.fit_transform(X_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score

import numpy as np

# Dictionary of models
models = {
    'Logistic Regression': LogisticRegression(penalty='l2', solver = 'newton-cg', class_weight='balanced', max_iter = 1000),
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree' : tree.DecisionTreeClassifier(class_weight = 'balanced'),
    'Random Forest' : RandomForestClassifier(n_estimators=300, class_weight = 'balanced'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=300),
    'AdaBoost' : AdaBoostClassifier(n_estimators=300),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Support Vector Machine': SVC(kernel = 'rbf', class_weight = 'balanced')
}

stratified_k_fold = StratifiedKFold(n_splits=5)

recall_scores = {}

for name, model in models.items():

    try:
        
        clf = model
        
        clf.fit(X_scaled, y_train)
        
        y_pred = clf.predict(X_scaled_test)

        recallScore = recall_score(y_test, y_pred)

        recall_scores[name] = []
        
        recall_scores[name].append(clf)
        
        recall_scores[name].append(recallScore)

        print(f"Macro Recall of {name}: {recallScore:.4f}")
    
    except Exception as e:

        print(f"Error evaluating {name}: {e}")